In [1]:
import io
import base64

from dash import Dash, dcc, html, Input, Output, no_update, callback
import plotly.express as px

from PIL import Image

data_x = [1,1,2,2]
data_y = [1,2,1,2]
fig = px.scatter(x=data_x, y=data_y)
fig.update_traces(
    hoverinfo="none",
    hovertemplate=None,
    marker=dict(size=30)
)
fig.update_layout(
    xaxis=dict(range=[-1,4]),
    yaxis=dict(range=[-1,4])
)

# Set up the app now
app = Dash(__name__)

app.layout = html.Div(
    className="container",
    children=[
        dcc.Graph(id="graph-2-dcc", figure=fig, clear_on_unhover=True),
        dcc.Tooltip(id="graph-tooltip-2", direction='bottom'),
    ],
)

@callback(
    Output("graph-tooltip-2", "show"),
    Output("graph-tooltip-2", "bbox"),
    Output("graph-tooltip-2", "children"),
    Output("graph-tooltip-2", "direction"),

    Input("graph-2-dcc", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update, no_update

    # Load image with pillow
    image_path = 'imagen.jpg'
    im = Image.open(image_path)

    # dump it to base64
    buffer = io.BytesIO()
    im.save(buffer, format="jpeg")
    encoded_image = base64.b64encode(buffer.getvalue()).decode()
    im_url = "data:image/jpeg;base64, " + encoded_image

    # demo only shows the first point, but other points may also be available
    hover_data = hoverData["points"][0]
    bbox = hover_data["bbox"]

    # control the position of the tooltip
    y = hover_data["y"]
    direction = "bottom" if y > 1.5 else "top"

    children = [
        html.Img(
            src=im_url,
            style={"width": "150px"},
        ),
        html.P("Image from base64 string"),
    ]

    return True, bbox, children, direction

if __name__ == "__main__":
    app.run(debug=True)


In [28]:
import plotly.express as px
df = px.data.iris() # iris is a pandas DataFrame

image_path = 'imagen.jpg'
im = Image.open(image_path)
buffer = io.BytesIO()
im.save(buffer, format="jpeg")
encoded_image = base64.b64encode(buffer.getvalue()).decode()
im_url = "data:image/jpeg;base64, " + encoded_image
df['url'] = im_url

fig = px.scatter(df, x="sepal_width", y="sepal_length", custom_data=["url"], text='url')
fig.update_traces(marker_size=10,hovertemplate = '<b>data</b><br>'+
                  '<img src="imagen.jpg" alt="Girl in a jacket" width="500" height="600">'+
                  '<br>Provincia: %{customdata[0]}')
fig.show()

In [11]:
import plotly.express as px
import plotly
from PIL import Image, ImageDraw

def add_corners(im, rad):
    circle = Image.new('L', (rad * 2, rad * 2), 0)
    draw = ImageDraw.Draw(circle)
    draw.ellipse((0, 0, rad * 2 - 1, rad * 2 - 1), fill=255)
    alpha = Image.new('L', im.size, 255)
    w, h = im.size
    alpha.paste(circle.crop((0, 0, rad, rad)), (0, 0))
    alpha.paste(circle.crop((0, rad, rad, rad * 2)), (0, h - rad))
    alpha.paste(circle.crop((rad, 0, rad * 2, rad)), (w - rad, 0))
    alpha.paste(circle.crop((rad, rad, rad * 2, rad * 2)), (w - rad, h - rad))
    im.putalpha(alpha)
    return im

img = ['imagen.jpg','imagen.jpg','imagen.jpg','imagen.jpg','imagen.jpg']
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])

for x,y, png in zip(fig.data[0].x, fig.data[0].y, img):
    im = Image.open(png)
    im = add_corners(im, 50)
    fig.add_layout_image(
        x=x,
        y=y,
        source=im,
        xref="x",
        yref="y",
        sizex=2,
        sizey=2,
        xanchor="center",
        yanchor="middle",
    )
plotly.offline.plot(fig,filename='data_image.html')
fig

In [42]:
import altair as alt
import pandas as pd

data = pd.DataFrame([dict(id=i/10) for i in range(1, 401)])

person = (
    "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
    "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
    "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
    "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
    "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
    "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
    "-0.6 -0.4 -0.6z"
)

alt.Chart(data).transform_calculate(
    row="ceil(datum.id/4)"
).transform_calculate(
    col="(datum.id - datum.row*4)"
).mark_point(
    filled=True,
    size=50
).encode(
    alt.X("col:O").axis(None),
    alt.Y("row:O").axis(None),
    alt.ShapeValue(person)
).properties(
    width=1400,
    height=800
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

In [39]:
data

,id
0,0.1
1,0.2
2,0.3
3,0.4
4,0.5
...,...
395,39.6
396,39.7
397,39.8
398,39.9
